In [ ]:
# Spark ML : Titanic

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('Spark DataFrames') \
    .getOrCreate()
sc = spark.sparkContext

In [2]:
titanic = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load("/demo/titanic/titanic-train.csv"))

In [3]:
display(titanic)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [4]:
titanic.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
titanic.describe()

DataFrame[summary: string, PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [7]:
titanic.describe().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [12]:
### GroupBy 
grouped = titanic.groupby("Sex")

### Grouped Average
grouped.avg().toPandas()

,Sex,avg(PassengerId),avg(Survived),avg(Pclass),avg(Age),avg(SibSp),avg(Parch),avg(Fare)
0,female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
1,male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


In [13]:
grouped.count().toPandas()

,Sex,count
0,female,314
1,male,577


In [14]:
### Aggregate multiple columns with different operations 
agg_dict={"Survived":"mean", "Fare":"sum"}
grouped.agg(agg_dict).show() #toPandas()

+------+-------------------+-----------------+
|   Sex|      avg(Survived)|        sum(Fare)|
+------+-------------------+-----------------+
|female| 0.7420382165605095|13966.66279999999|
|  male|0.18890814558058924|14727.28649999999|
+------+-------------------+-----------------+



In [15]:
### SELECT Columns 
from pyspark.sql import functions as F 

### select missing values
missing_values_cabin = titanic.select(F.sum(F.col("Cabin").isNull().cast("int")))
missing_values_cabin.show()

+---------------------------------+
|sum(CAST((Cabin IS NULL) AS INT))|
+---------------------------------+
|                              687|
+---------------------------------+



In [16]:
### Alias 
titanic.select(F.sum("Fare").alias("Total Fare")).show()

+------------------+
|        Total Fare|
+------------------+
|28693.949299999967|
+------------------+



In [18]:
### Remove null values 
titanic.where((~F.col("Cabin").isNull()) & (~F.col("Embarked").isNull())).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
197,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
198,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
199,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
200,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [19]:
titanic.where((F.col("Cabin").isNull())).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
2,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
3,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
682,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,None,S
683,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,None,Q
684,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S
685,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,NaN,1,2,W./C. 6607,23.4500,None,S


**On n'a rien fait car on n'a jamais enregistré le résultat !**

**Exercice:**

Reprendre les opérations de façon à sauvegarder ce dataset modifié.


In [20]:
!hdfs dfs -rm /stagiaire/XXXX/titanic-clean/*
!hdfs dfs -rmdir /stagiaire/XXXX/titanic-clean

rm: `/stagiaire/XXXX/titanic-clean/*': No such file or directory
rmdir: `/stagiaire/XXXX/titanic-clean': No such file or directory


In [21]:
titanic.write.csv("/titanic-clean")

In [ ]:
!hdfs dfs -ls /stagiaire/XXXX/titanic-clean

In [ ]:
titanic2 = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load("/stagiaire/XXXX/titanic-clean/*.csv"))

In [ ]:
titanic2.head(3)

In [ ]:
sc.stop()